In [ ]:
from mainnet_launch.database.schema.full import *
from mainnet_launch.database.schema.postgres_operations import *
import plotly.express as px

from mainnet_launch.constants import *

destinations_df = get_full_table_as_df(Destinations, where_clause=Destinations.chain_id == 1)
blocks_df = get_full_table_as_df(Blocks, where_clause=Blocks.chain_id == 1)
event_df = get_full_table_as_df(RebalanceEvents)
plan_df = get_full_table_as_df(RebalancePlans)
trasactions_df = get_full_table_as_df(Transactions)
destination_tokens_df = get_full_table_as_df(DestinationTokens)

2025-05-30 09:00:35,532 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-30 09:00:35,534 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-30 09:00:35,535 INFO sqlalchemy.engine.Engine [cached since 917s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x1400ed6f0>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-30 09:00:35,536 INFO sqlalchemy.engine.Engine 
            SELECT *
            FROM destinations
            WHERE destinations.chain_id = 1
        
2025-05-30 09:00:35,538 

In [ ]:
destination_to_underlying = destinations_df.set_index("destination_vault_address")["underlying_symbol"].to_dict()
destination_to_underlying

{'0xE54173ba36b507fd6487D21d1d78762B03303f18': 'sUSDEsUSDS',
 '0x04ee6c666a01018a070925040ea2aF6554d051e4': 'MtEthena',
 '0xA1b9297525388BA3985413D982863f31e2A29d05': 'ECLP-wstETH-wETH',
 '0x9906eB64BA32Fb4fD3e5541ecA95e57610084d02': 'crvUSDFRAX-f',
 '0x5Ff56531f92f1e3f2bE616760F05e35caE630D91': 'wETHrETH',
 '0x41A8DFc857554aE23A4b31188a3109EA4e2377D3': 'crv2pool',
 '0x35911af1B570E26f668905595dEd133D01CD3E5a': 'dineroETH',
 '0xE0649C82BBfF3C8F5FE000CdA5062c55c051B2EE': 'sUSDEsUSDS',
 '0x8acA8accFb69adeff607431E0F25466B7b76a8Ad': 'scrvUSD',
 '0xE800e3760FC20aA98c5df6A9816147f190455AF3': 'autoLRT',
 '0x1E581AD9ef141C3037991806383B7990c1f68312': 'B-rETH-STABLE',
 '0xa767F35BF1d51f4ab7432188DfE6324934c9CCbD': 'sUSDS',
 '0xdfE3fA7027E84f59b266459C567278C79fe86f0C': 'ETHx/wstETH',
 '0x2f2CC1bf461413014741dD68481dB4a3686DAC3D': 'ezETH-WETH-BPT',
 '0xC001f23397dB71B17602Ce7D90a983Edc38DB0d1': 'ethx-f',
 '0x28109255F49f0A02BB31358B3e0b5cb7512EB4eC': 'OETH/WETH',
 '0xAd7b8dB1efEe849c67060c6b10B

In [41]:
rebalance_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            RebalanceEvents,
        ),
        TableSelector(Transactions, join_on=(Transactions.tx_hash == RebalanceEvents.tx_hash)),
        TableSelector(
            Blocks,
            [Blocks.datetime],
            (Transactions.block == Blocks.block) & (Transactions.chain_id == Blocks.chain_id),
        ),
    ],
    where_clause=(RebalanceEvents.autopool_vault_address == AUTO_USD.autopool_eth_addr),
    order_by=Blocks.datetime,
)

tokens_df = merge_tables_as_df(
    [
        TableSelector(
            DestinationTokens,
        ),
        TableSelector(
            table=Tokens,
            select_fields=[Tokens.symbol],
            join_on=(DestinationTokens.token_address == Tokens.token_address),
        ),
    ],
)


tokens_df

2025-05-30 09:17:50.934 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 09:17:50.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 09:17:50.936 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 09:17:50.937 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 09:17:50.937 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 09:17:50.938 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 09:17:50.939 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 09:17:50.939 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

,destination_vault_address,chain_id,token_address,index,symbol
0,0x04ee6c666a01018a070925040ea2aF6554d051e4,1,0x83F20F44975D03b1b09e64809B757c47f942BEeA,0,sDAI
1,0x2f2CC1bf461413014741dD68481dB4a3686DAC3D,1,0xbf5495Efe5DB9ce00f80364C8B423567e58d2110,0,ezETH
2,0x5Ff56531f92f1e3f2bE616760F05e35caE630D91,1,0xae78736Cd615f374D3085123A210448E74Fc6393,1,rETH
3,0x53C8C211350b635269e02B2C2f1077b850C657dB,1,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,0,scrvUSD
4,0x65efCF2cce562DCBf07e805eEbeDeF21Dbd8Ea3D,1,0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E,1,crvUSD
...,...,...,...,...,...
224,0xBd137c56f3116E5c36753037a784FF844F84F59c,8453,0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22,0,cbETH
225,0xb3723E69D6e4553239d7dAf514bF8ff7998500E0,8453,0x04C0599Ae5A44757c0af6F9eC3b93da8976c150A,0,weETH
226,0xb3723E69D6e4553239d7dAf514bF8ff7998500E0,8453,0x4200000000000000000000000000000000000006,1,WETH
227,0x58C2233399B85b53C5506f78eAaae9b0DBA1eD3E,8453,0x4200000000000000000000000000000000000006,0,WETH


In [65]:
destination_to_underlying = destinations_df.set_index("destination_vault_address")["underlying_symbol"].to_dict()

rebalance_df["destination_in_symbol"] = rebalance_df["destination_in"].map(destination_to_underlying)
rebalance_df["destination_out_symbol"] = rebalance_df["destination_out"].map(destination_to_underlying)
rebalance_df["move_name"] = rebalance_df["destination_out_symbol"] + " -> " + rebalance_df["destination_in_symbol"]


destination_token_address_to_symbols = (
    tokens_df.groupby("destination_vault_address")["symbol"].apply(tuple).apply(str).to_dict()
)
rebalance_df["destination_in_tokens"] = rebalance_df["destination_in"].map(destination_token_address_to_symbols)
rebalance_df["destination_out_tokens"] = rebalance_df["destination_out"].map(destination_token_address_to_symbols)
rebalance_df["tokens_move_name"] = (
    rebalance_df["destination_out_tokens"] + " -> " + rebalance_df["destination_in_tokens"]
)


rebalance_df["spot_swap_cost"] = rebalance_df["spot_value_in"] - rebalance_df["spot_value_out"]
rebalance_df["slippage_bps"] = 10_000 * rebalance_df["spot_swap_cost"] / rebalance_df["spot_value_out"]


fig = px.bar(
    rebalance_df.groupby("move_name")["spot_swap_cost"].sum().sort_values(),
    height=800,
)

fig.update_layout(
    height=600, margin=dict(t=50, b=150, l=50, r=50), xaxis_tickangle=-75  # tilt labels so they fit better
)

In [66]:
fig = px.bar(
    100
    * rebalance_df.groupby("move_name")["spot_value_out"].sum().sort_values()
    / rebalance_df["spot_value_out"].sum(),
    height=800,
)

fig.update_layout(
    height=600, margin=dict(t=50, b=150, l=50, r=50), xaxis_tickangle=-80  # tilt labels so they fit better
)
fig.update_layout(margin=dict(b=250))

In [73]:
fig = px.bar(
    100
    * rebalance_df.groupby("tokens_move_name")["spot_value_out"].sum().sort_values()
    / rebalance_df["spot_value_out"].sum(),
    height=800,
)

fig.update_layout(
    height=600, margin=dict(t=50, b=150, l=50, r=50), xaxis_tickangle=-80  # tilt labels so they fit better
)
fig.update_layout(margin=dict(b=250))

In [75]:
fig = px.bar(
    10_000
    * (
        rebalance_df.groupby("tokens_move_name")["spot_swap_cost"].sum()
        / rebalance_df.groupby("tokens_move_name")["spot_value_out"].sum()
    ).sort_values(),
    height=800,
)

fig.update_layout(
    height=600, margin=dict(t=50, b=150, l=50, r=50), xaxis_tickangle=-80  # tilt labels so they fit better
)
fig.update_layout(margin=dict(b=250))

In [78]:
fig = px.bar(
    100 * rebalance_df.groupby("tokens_move_name")["spot_value_out"].sum() / rebalance_df["spot_value_out"].sum(),
    height=800,
)

fig.update_layout(
    height=600, margin=dict(t=50, b=150, l=50, r=50), xaxis_tickangle=-80  # tilt labels so they fit better
)
fig.update_layout(margin=dict(b=250))

In [80]:
fig = px.bar(
    rebalance_df.groupby("tokens_move_name")["spot_swap_cost"].sum().sort_values(),
    height=800,
)

fig.update_layout(
    height=600, margin=dict(t=50, b=150, l=50, r=50), xaxis_tickangle=-80  # tilt labels so they fit better
)
fig.update_layout(margin=dict(b=250))